In [ ]:
pip install pulp


     |████████████████████████████████| 40.6MB 104kB/s 


In [ ]:
from pulp import *

In [ ]:
modelo = LpProblem("ProblemaMRP", LpMinimize )

In [ ]:
#Definición de conjuntos
Items = range(1,5) # Hay 4 materias, item 1: Producto final, item 2: A, item 3: B, item 4: C
Periodos = range(-4,6)
Centros_Trabajo = range(1,5) # 

#Definición de parámetros
#Demanda del producto i en el periodo t >= 0
d = {1:{0:15,1:25,2:40,3:40,4:20,5:20},
     2:{0:15,1:25,2:40,3:40,4:20,5:20},
     3:{0:15,1:25,2:40,3:40,4:20,5:20},
     4:{0:15,1:25,2:40,3:40,4:20,5:20}}
#Tiempo unitario de operacion del item i en la estación k
#Estación 1: Corte
#Estación 2: Transferencia
#Estación 3: Ensamble
#Estación 4: Pintado
alfa = {1:1.5,2:0, 3:0.5,4:2.5}
#Tiempo de setup de la estación k
beta = {1:25,2:20,3:10,4:30}
#Tiempo máximo de trabajo en cada estación
L = {1:480,2:480,3:960,4:480}
#Tiempo de entrega
#item 1: PF
#item 2: A
#item 3: B
#item 4: C
gamma = {1:0,2:2,3:1,4:2}
#Relación entre items i,j
r = {1:{1:0,2:0,3:0,4:0},
     2:{1:1,2:0,3:0,4:0},
     3:{1:0,2:2,3:0,4:0},
     4:{1:1,2:3,3:0,4:0}}

#M grande
M = 100000

#Definición de variables
x = LpVariable.dicts("x",(Items,Periodos), lowBound= 0, cat ='Integer')
s = LpVariable.dicts("s",(Items,Periodos), lowBound= 0, cat ='Integer')
y = LpVariable.dicts("y",(Items,Periodos), cat ='Binary')

#Función Objetivo
modelo += lpSum(100*x[1][t] + 0.5*s[1][t] + 30*y[2][t] for t in Periodos)

#Restricciones
for t in Periodos:
  if (t >= 0):
    modelo += s[1][t-1] + x[1][t] ==  d[1][t] + s[1][t]

for t in Periodos:
  if ( t <= 0):
    modelo += s[1][t] == 0 #Me debo asegurar que el nivel de inventario para t = 0 empiece desde 0

for i in Items:
  for t in Periodos:
    if ((i >= 2) and ( t >= -2)):
      modelo += x[i][t-gamma[i]] == lpSum(r[i][j]*x[j][t] for j in Items)

for t in Periodos:
  if (t >= -2):
    modelo += x[2][t] <= M*y[2][t]

for k in  Centros_Trabajo:
  for t in Periodos:
    if t>= -2:
      modelo += alfa[k]*x[2][t] + beta[k]*y[2][t] <= L[k]

modelo.solve()






















    

/usr/local/lib/python3.6/dist-packages/pulp/pulp.py:1521: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


PulpSolverError: ignored